## Char-RNN

In [1]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Sequential
tf.compat.v1.enable_eager_execution()
tf.compat.v1.disable_v2_behavior()
import tensorflow.experimental.numpy as tnp
tf.enable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
lyrics = open("Lyrics_Counterparts.txt", encoding="utf-8").read().lower()

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(lyrics)

In [4]:
tokenizer.texts_to_sequences(["First"])

[[17, 5, 8, 9, 3]]

In [5]:
max_id = len(tokenizer.word_index) #number of distinct characters in text
dataset_size = tokenizer.document_count #total number of characters
print(max_id, " ", dataset_size)

47   70652


In [6]:
[encoded] = np.array(tokenizer.texts_to_sequences([lyrics])) - 1
#encodes the full text so each character is represented by its ID

In [7]:
#splitting dataset for training/test/validation
train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
#convert long string of characters into windows for training(batches)
n_steps = 100
window_length = n_steps + 1 #target character = input shifted 1 char ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
#flatten the dataset for training
dataset = dataset.flat_map(lambda window: window.batch(window_length))
#this gives us a single tensor for each window

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [11]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [12]:
dataset = dataset.prefetch(1)

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                    dropout=0.3, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.3, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.3, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.3, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50)

Epoch 1/50
1984/1984 [==============================] - 2148s 1s/step - loss: 2.3424
Epoch 2/50
1984/1984 [==============================] - 2034s 1s/step - loss: 1.5608
Epoch 3/50
1984/1984 [==============================] - 2058s 1s/step - loss: 1.4299
Epoch 4/50
1984/1984 [==============================] - 2044s 1s/step - loss: 1.3675
Epoch 5/50
1984/1984 [==============================] - 2135s 1s/step - loss: 1.3339
Epoch 6/50
1984/1984 [==============================] - 2063s 1s/step - loss: 1.3091
Epoch 7/50
1984/1984 [==============================] - 2053s 1s/step - loss: 1.2917
Epoch 8/50
1984/1984 [==============================] - 2058s 1s/step - loss: 1.2779
Epoch 9/50
1984/1984 [==============================] - 2054s 1s/step - loss: 1.2656
Epoch 10/50
1984/1984 [==============================] - 2055s 1s/step - loss: 1.2563
Epoch 11/50
1984/1984 [==============================] - 2066s 1s/step - loss: 1.2491
Epoch 12/50
1984/1984 [==============================] - 2059s 

In [14]:
model.save('char_rnn_lyrics')

In [15]:
tf.executing_eagerly()

True

In [16]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [17]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new,steps=1)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba)/ temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
#this function picks the next character randomly, with a probability equal to the estimated probability,using tf.random.categorical
#categorical() samples random class indices, given the class probs (logits)
#temperature is a variables that controls the diversity of the characters generated
# a near-zero temperature will output characters with high probabilities
# a high temperature will give all characters equal probability

In [18]:
#this function repeatedly calls the above function to generate text
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char (text, temperature)
    return text

In [19]:
print(complete_text("m",temperature=1)+ complete_text("i",temperature=1)+ complete_text("t",temperature=1)+complete_text("c",temperature=1)+complete_text("h",temperature=1)) 

mind they have burdened
we will be remembered
we will be remembered
we will be remembered
we will be inth back of alive
and i won’t life is your feeling there is right? 
and i am an outcast
i am not leated than expression

existence so this words lie to shake a north the blight of open arms
though i facan fame we're never being them
and while i made we are left to relect from the tells
that we've beenhe existence of crashing to be
and i will never being think
i reject me far anyone close words
but i 


## Stateful RNN

In [20]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=n_steps, drop_remainder=True)

In [21]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [22]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [23]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.3, recurrent_dropout=0.3, batch_input_shape=[batch_size, None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.3, recurrent_dropout=0.3),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.3, recurrent_dropout=0.3),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.3, recurrent_dropout=0.3),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])

In [24]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [25]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps

In [26]:
history = model.fit(dataset, epochs=500, callbacks=[ResetStatesCallback()], steps_per_epoch=steps_per_epoch)

Epoch 1/500
19/19 [==============================] - 22s 906ms/step - loss: 3.5794
Epoch 2/500
19/19 [==============================] - 18s 964ms/step - loss: 3.0053
Epoch 3/500
19/19 [==============================] - 18s 950ms/step - loss: 2.9800
Epoch 4/500
19/19 [==============================] - 18s 944ms/step - loss: 2.9741
Epoch 5/500
19/19 [==============================] - 18s 925ms/step - loss: 2.9711
Epoch 6/500
19/19 [==============================] - 18s 928ms/step - loss: 2.9586
Epoch 7/500
19/19 [==============================] - 18s 935ms/step - loss: 2.9077
Epoch 8/500
19/19 [==============================] - 18s 926ms/step - loss: 2.8445
Epoch 9/500
19/19 [==============================] - 18s 937ms/step - loss: 2.7682
Epoch 10/500
19/19 [==============================] - 18s 934ms/step - loss: 2.6967
Epoch 11/500
19/19 [==============================] - 18s 925ms/step - loss: 2.6279
Epoch 12/500
19/19 [==============================] - 18s 924ms/step - loss: 2.5652
E

19/19 [==============================] - 18s 926ms/step - loss: 1.6805
Epoch 99/500
19/19 [==============================] - 18s 947ms/step - loss: 1.6761
Epoch 100/500
19/19 [==============================] - 18s 939ms/step - loss: 1.6780
Epoch 101/500
19/19 [==============================] - 18s 925ms/step - loss: 1.6782
Epoch 102/500
19/19 [==============================] - 18s 955ms/step - loss: 1.6632
Epoch 103/500
19/19 [==============================] - 19s 1s/step - loss: 1.6736
Epoch 104/500
19/19 [==============================] - 18s 937ms/step - loss: 1.6586
Epoch 105/500
19/19 [==============================] - 18s 929ms/step - loss: 1.6594
Epoch 106/500
19/19 [==============================] - 18s 932ms/step - loss: 1.6572
Epoch 107/500
19/19 [==============================] - 18s 934ms/step - loss: 1.6570
Epoch 108/500
19/19 [==============================] - 18s 934ms/step - loss: 1.6528
Epoch 109/500
19/19 [==============================] - 18s 925ms/step - loss: 1.652

19/19 [==============================] - 18s 938ms/step - loss: 1.5195
Epoch 195/500
19/19 [==============================] - 18s 951ms/step - loss: 1.5195
Epoch 196/500
19/19 [==============================] - 18s 931ms/step - loss: 1.5226
Epoch 197/500
19/19 [==============================] - 18s 935ms/step - loss: 1.5202
Epoch 198/500
19/19 [==============================] - 18s 961ms/step - loss: 1.5245
Epoch 199/500
19/19 [==============================] - 18s 935ms/step - loss: 1.5127
Epoch 200/500
19/19 [==============================] - 18s 941ms/step - loss: 1.5226
Epoch 201/500
19/19 [==============================] - 18s 942ms/step - loss: 1.5120
Epoch 202/500
19/19 [==============================] - 18s 949ms/step - loss: 1.5201
Epoch 203/500
19/19 [==============================] - 18s 936ms/step - loss: 1.5135
Epoch 204/500
19/19 [==============================] - 19s 997ms/step - loss: 1.5088
Epoch 205/500
19/19 [==============================] - 18s 935ms/step - loss: 1

19/19 [==============================] - 18s 958ms/step - loss: 1.4616
Epoch 291/500
19/19 [==============================] - 18s 925ms/step - loss: 1.4609
Epoch 292/500
19/19 [==============================] - 18s 930ms/step - loss: 1.4588
Epoch 293/500
19/19 [==============================] - 17s 922ms/step - loss: 1.4653
Epoch 294/500
19/19 [==============================] - 18s 930ms/step - loss: 1.4607
Epoch 295/500
19/19 [==============================] - 17s 917ms/step - loss: 1.4529
Epoch 296/500
19/19 [==============================] - 18s 924ms/step - loss: 1.4579
Epoch 297/500
19/19 [==============================] - 17s 911ms/step - loss: 1.4535
Epoch 298/500
19/19 [==============================] - 18s 940ms/step - loss: 1.4516
Epoch 299/500
19/19 [==============================] - 17s 914ms/step - loss: 1.4569
Epoch 300/500
19/19 [==============================] - 18s 934ms/step - loss: 1.4496
Epoch 301/500
19/19 [==============================] - 18s 938ms/step - loss: 1

19/19 [==============================] - 18s 943ms/step - loss: 1.4189
Epoch 387/500
19/19 [==============================] - 18s 954ms/step - loss: 1.4266
Epoch 388/500
19/19 [==============================] - 18s 926ms/step - loss: 1.4208
Epoch 389/500
19/19 [==============================] - 18s 932ms/step - loss: 1.4271
Epoch 390/500
19/19 [==============================] - 18s 925ms/step - loss: 1.4266
Epoch 391/500
19/19 [==============================] - 18s 957ms/step - loss: 1.4205
Epoch 392/500
19/19 [==============================] - 18s 936ms/step - loss: 1.4237
Epoch 393/500
19/19 [==============================] - 18s 924ms/step - loss: 1.4181
Epoch 394/500
19/19 [==============================] - 18s 929ms/step - loss: 1.4179
Epoch 395/500
19/19 [==============================] - 18s 929ms/step - loss: 1.4312
Epoch 396/500
19/19 [==============================] - 17s 914ms/step - loss: 1.4291
Epoch 397/500
19/19 [==============================] - 18s 924ms/step - loss: 1

19/19 [==============================] - 17s 914ms/step - loss: 1.4017
Epoch 483/500
19/19 [==============================] - 18s 941ms/step - loss: 1.4034
Epoch 484/500
19/19 [==============================] - 18s 922ms/step - loss: 1.4028
Epoch 485/500
19/19 [==============================] - 18s 919ms/step - loss: 1.3999
Epoch 486/500
19/19 [==============================] - 17s 921ms/step - loss: 1.3993
Epoch 487/500
19/19 [==============================] - 18s 940ms/step - loss: 1.4018
Epoch 488/500
19/19 [==============================] - 18s 928ms/step - loss: 1.4008
Epoch 489/500
19/19 [==============================] - 17s 920ms/step - loss: 1.3968
Epoch 490/500
19/19 [==============================] - 18s 926ms/step - loss: 1.4001
Epoch 491/500
19/19 [==============================] - 18s 928ms/step - loss: 1.3915
Epoch 492/500
19/19 [==============================] - 17s 916ms/step - loss: 1.4007
Epoch 493/500
19/19 [==============================] - 18s 942ms/step - loss: 1

In [27]:
model.save('stateful_rnn_lyrics')

In [28]:
tf.executing_eagerly()

True

In [29]:
#stateless copy to allow us to use different batch sizes
stateless_model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [30]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [31]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [32]:
print(complete_text("t"))

tions starm
if won’t left to finden un a sleaved world
is all to you can’t exmecting the untray
to sk


In [33]:
print(complete_text("m")+ complete_text("i")+ complete_text("t")+complete_text("c")+complete_text("h")) 

ming for a share that but to leave
not what i know where i'm remembered in nothing

man if i am
promiing the insidence
decenting shearting in the rest back
our part
befring my heart both sleep the gracets carry of an used of wastes myself coure
so i am the gracious bfoint of the nust become those will ce on the thurs this alses of my mind
be searching the world failly greave
our lost in the composen bher in the stay
not know into myself direcussed that's we draved
drown ins alone of this


sports you
